Labels:
- StudyDate_DICOM
- PatientBirth
- PatientSex_DICOM
- ViewPosition_DICOM
- Projection
- Modality_DICOM
- LabelsLocalizationsBySentence


In [7]:
import pandas as pd

labels = pd.read_csv('PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv', index_col=0)

# Calculate age, If PatientBirth is NA, age is NA otherwise age is year - birth
labels['birth'] = labels['PatientBirth']
labels['year'] = labels['StudyDate_DICOM'].astype(str).str[:4].astype(int)
labels['age'] = (labels['year'] - labels['birth']).astype('Int64')

# Generate prompts
labels['age_prompt'] = labels['age'].apply(lambda x: '' if pd.isna(x) else 'age ' + str(x))
labels['sex_prompt'] = labels['PatientSex_DICOM'].apply(lambda x: ' male, ' if x == 'M' else ' female, ' if x == 'F' else ', ')
labels['view_prompt'] = labels['ViewPosition_DICOM'].apply(lambda x: '' if pd.isna(x) else 'view ' + str(x) + ', ')
labels['projection_prompt'] = labels['Projection'].apply(lambda x: '' if pd.isna(x) else 'projection ' + str(x) + ', ')
labels['modality_prompt'] = labels['Modality_DICOM'].apply(lambda x: '' if pd.isna(x) else 'modality ' + str(x) + ', ')
labels['diagnosis_prompt'] = labels['LabelsLocalizationsBySentence'].apply(lambda x: '' if pd.isna(x) else 'diagnosis ' + str(x))
labels['Prompt'] = 'Chest X-ray, ' + labels['age_prompt'] + labels['sex_prompt'] + labels['view_prompt'] + labels['projection_prompt'] + labels['modality_prompt'] + labels['diagnosis_prompt']
# remove square brackets and single quotes
labels['Prompt'] = labels['Prompt'].str.replace('[', '').str.replace(']', '').str.replace("'", '')
# Save a csv file with 3 columns, ImageID, ImageDir, and Prompt
labels[['ImageID', 'ImageDir', 'Prompt']].to_csv('PADCHEST_Prompt.csv', index=False)

# Print the ImageDir, ImageID, number of words and prompts for prompts with more than 75 words without truncation
labels['num_words'] = labels['Prompt'].str.split().str.len()
labels[['ImageDir', 'ImageID', 'num_words', 'Prompt']][labels['num_words'] > 75].reset_index(drop=True).style.set_properties(subset=['Prompt'], **{'width': '1000px'})

,ImageDir,ImageID,num_words,Prompt
0,4,322621863245407630024437930897524688767_2mj8aa.png,80,"Chest X-ray, age 62 male, projection L, modality CR, diagnosis atelectasis, heart insufficiency, pleural effusion, increased density, pneumonia, loc basal, loc pleural, loc right, loc cardiac, loc left, increased density, loc hilar, loc right, increased density, loc hilar, loc right, pleural effusion both sides, loc pleural, loc bilateral, pleural effusion both sides, loc pleural, loc bilateral, interstitial pattern, loc bilateral, exclude, superior mediastinal enlargement, supra aortic elongation, goiter, loc supra aortic, loc mediastinum, loc superior mediastinum, mediastinal enlargement, loc mediastinum"
1,5,303911634260682037750306148922413316981_4ufdzm.png,87,"Chest X-ray, age 92 female, view LL, projection L, modality DX, diagnosis aortic elongation, aortic atheromatosis, loc aortic, laminar atelectasis, loc basal, loc left, osteopenia, scoliosis, loc dorsal vertebrae, loc bone, COPD signs, hiatal hernia, hiatal hernia, humeral fracture, loc bone, loc left, loc humerus, callus rib fracture, loc rib, loc left, loc posterior rib, unchanged, vascular hilar enlargement, loc hilar, loc cardiac, subacromial space narrowing, loc right, loc rotator cuff, costophrenic angle blunting, loc right costophrenic angle, loc costophrenic angle, fibrotic band, chronic changes, loc pleural"
2,5,303911634260682037750306148922413316981_4uc8jd.png,87,"Chest X-ray, age 92 female, view PA, projection PA, modality DX, diagnosis aortic elongation, aortic atheromatosis, loc aortic, laminar atelectasis, loc basal, loc left, osteopenia, scoliosis, loc dorsal vertebrae, loc bone, COPD signs, hiatal hernia, hiatal hernia, humeral fracture, loc bone, loc left, loc humerus, callus rib fracture, loc rib, loc left, loc posterior rib, unchanged, vascular hilar enlargement, loc hilar, loc cardiac, subacromial space narrowing, loc right, loc rotator cuff, costophrenic angle blunting, loc right costophrenic angle, loc costophrenic angle, fibrotic band, chronic changes, loc pleural"
3,6,311480822391073525044015832605252159955_ouorsn.png,94,"Chest X-ray, age 90 male, view AP, projection AP, modality DX, diagnosis goiter, superior mediastinal enlargement, loc mediastinum, loc superior mediastinum, aortic elongation, aortic atheromatosis, loc aortic, laminar atelectasis, loc basal bilateral, exclude, scoliosis, vertebral degenerative changes, loc bone, COPD signs, hiatal hernia, hiatal hernia, calcified granuloma, loc bilateral, osteosynthesis material, loc humeral head, loc left, loc humerus, callus rib fracture, loc rib, loc right, alveolar pattern, pneumonia, loc basal bilateral, pacemaker, single chamber device, loc left, fibrotic band, chronic changes, loc pleural, hilar congestion, loc hilar, loc cardiac, apical pleural thickening, loc pleural"
4,9,174654958803616926510759514339561596940_xze55j.png,77,"Chest X-ray, age 78 male, projection L, modality CR, diagnosis laminar atelectasis, bronchiectasis, loc middle lobe, loc bronchi, vertebral degenerative changes, air trapping, loc retrocardiac, calcified densities, granuloma, loc right upper lobe, loc middle lung field, pseudonodule, loc middle lung field, normal, suboptimal study, loc right, soft tissue mass, loc soft tissue, loc rib, loc right, exclude, loc cervical, chronic changes, chronic changes, apical pleural thickening, fibrotic band, loc pleural, apical pleural thickening, fibrotic band, loc pleural"
5,9,174654958803616926510759514339561596940_xjxwm2.png,77,"Chest X-ray, age 78 male, projection PA, modality CR, diagnosis laminar atelectasis, bronchiectasis, loc middle lobe, loc bronchi, vertebral degenerative changes, air trapping, loc retrocardiac, calcified densities, granuloma, loc right upper lobe, loc middle lung field, pseudonodule, loc middle lung field, normal, suboptimal study, loc right, soft tissue mass, loc soft tissue, loc rib, loc right, exclude, loc cervical, chronic changes, chronic changes,

In [8]:
# Print the number of each "Modality_DICOM" grouped b

In [9]:
import os
path = 'test'
for file in os.listdir(path=path):
    # for each file in the directory, save a .txt file with the corresponding Prompt and the same name as the image file wihtout .png
    print(labels[labels['ImageID'] == file]['Prompt'].values[0])
    print(labels[labels['ImageID'] == file]['Prompt'].values[0], file=open(os.path.join(path, file.split(".")[0]) + '.txt', 'w'))

Chest X-ray, age 72 male, view POSTEROANTERIOR, projection PA, modality CR, diagnosis COPD signs, COPD signs, COPD signs, nipple shadow, loc lower lung field, loc nipple, loc left, hemidiaphragm elevation right side, loc diaphragm, loc right
Chest X-ray, age 72 male, view LATERAL, projection L, modality CR, diagnosis COPD signs, COPD signs, COPD signs, nipple shadow, loc lower lung field, loc nipple, loc left, hemidiaphragm elevation right side, loc diaphragm, loc right
Chest X-ray, age 81 female, view LATERAL, projection L, modality CR, diagnosis exclude, exclude, exclude, mass, pulmonary mass, loc right upper lobe, loc hemithorax
Chest X-ray, age 81 female, view POSTEROANTERIOR, projection PA, modality CR, diagnosis exclude, exclude, exclude, mass, pulmonary mass, loc right upper lobe, loc hemithorax
Chest X-ray, age 28 male, projection AP_horizontal, modality CR, diagnosis alveolar pattern, pleural effusion, loc pleural, loc right, loc middle lobe, loc left, loc retrocardiac, pneumo